# Experimentation

This notebook's purpose is to easily experiment some chunks of code. There will be nothing here that actually works. The main purpose is to show my idea walkthourgs. When time passes I am expecting to copy-paste the working code from here to original .py script files.

## [Read screen from a web page experiments](https://github.com/hakanonal/geodashml/projects/1#card-37027228)

26.04.2020 - I have initally googled and bumped into [this](https://stackoverflow.com/questions/38568804/python-open-html-file-take-screenshot-crop-and-save-as-image) and [this](https://stackoverflow.com/questions/38568804/python-open-html-file-take-screenshot-crop-and-save-as-image). 

- Some fooling around with these ideas. 
- it appears that selenium is the right tool to get the screen shot. I still not sure if the scracth version of the game is going to provide the continoues images during playing the game. After all I do not have any idea what the game's runtime. we need to test more if we can get screenshots in a loop so that we can get series of screenshot images to feed into the network.
- I have bumped into selenium installation problem, currentlly trying to resolve, "Message: 'Google\ Chrome.app' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home" error message 
- This [page](https://www.selenium.dev/documentation/en/webdriver/driver_requirements/) seems to be helped a lot. It is the official selenium documentaion page that explains working principle and driver installation. It also re-directs to google's own [page](https://sites.google.com/a/chromium.org/chromedriver/downloads) to download the driver. I have matched my installed chrome application version with the driver version. I have also needed my mac security setting to allow un-trusted code to execute. I have moved it to /usr/local/bin folder. I choose to trust [this](https://sites.google.com/a/chromium.org/chromedriver/home) guys. Consider [security](https://sites.google.com/a/chromium.org/chromedriver/security-considerations) also.

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=options)

driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
image = driver.find_element_by_id('view').screenshot_as_png

- selenium seems to be working. Now we need if we can get the image that we want. It seems not. It is not rendered properlly.

In [ ]:
from IPython.display import Image
Image(image)

27.04.2020 - Continue to explore selenium api
- after a blank page I will try different elements to visualize, there may be problem with the flash or html5 or javascript (non-static) elements.
- started to read selenium [webdriver documantation](https://www.selenium.dev/documentation/en/webdriver/waits/) in detail. "Waits" section seems to be promissing
- using the WebDriverWait api has solved the problem.
- waiting is solved the problem but it works inconsistantly. after wating I may need to call find_element function second time. I am planing to fool arround with this code more.
- driver.quit() call seems to be important, otherwise the application stays open. so I have put it into try: finally block. to execute every time.
- trying to understand which element to wait for specificlly this project
- to watch the progress we can get rid of "options.add_argument('headless')" statement so a new window will open
- so after fittling around a little bit more, I have finnally clicked the green flag element an get to the main menu, now I plan to start game and get the updated series images in a loop.
- I am trying to find the right element to start the game by clicking. can not find where to click
- I have founded the element that I have trying to find using find_element**s**_by_css_selector which returns a list. however find_element_by_css_selector this version brings only one element.
- The selenium api is working perfectlly fine. My problem is in scratch. Normally user clicks on the head of the level name or even pressing space is enough, however I can not which element is actually catching that key stroke. I need to find a way to debug the scratch anc find the element that catches the key presses.
- In scratch all visible game elements are in a single element tag named as canvas. I have tried to press space all the parent elements of the canvas however I always get the following error message "Message: element not interactable"
- IS it possible to overcome this problem by getting the root docuement try to send the key command? Checking more on selenium documents. Yes it did work. find element by tag and the tag name is "html" Yeah!
- Now I can consantrate on getting series of images and series of sending keys...
- Just to see if it is working I have waited 1 second and take 3 different snapshots. You can see that our fellow moved forward on the last image.

In [ ]:
from selenium import webdriver
from IPython.display import Image
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=options)

try:
    driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
    wait = WebDriverWait(driver,timeout=10)
    wait.until(lambda d: d.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")) #wait until green flag appears

    flag_element = driver.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")
    flag_element.click()

    wait.until(lambda d: d.find_element_by_css_selector(".monitor_monitor-container_2J9gl")) #wait until score board appears

    driver.implicitly_wait(10)
    html = driver.find_element_by_tag_name("html")
    html.send_keys(Keys.SPACE)
    wait.until(lambda d: d.find_element_by_css_selector(".monitor_value_3Yexa")) #wait until score monitor element appears. 

    image0 = driver.find_element_by_css_selector("#view").screenshot_as_png
    driver.implicitly_wait(1)
    image1 = driver.find_element_by_css_selector("#view").screenshot_as_png
    driver.implicitly_wait(1)
    image2 = driver.find_element_by_css_selector("#view").screenshot_as_png
    driver.implicitly_wait(1)


finally:
    #print("ok")
    driver.quit()

Image(image0)
Image(image1)
Image(image2)

28.04.2020 - Let's put this into loop so that we can get series of images.

- that was easy. using [this](https://stackoverflow.com/questions/11854847/how-can-i-display-an-image-from-a-file-in-jupyter-notebook)
- ok then we need this image to be numpy array. go...
- I have checked [this](https://stackoverflow.com/questions/57318892/convert-base64-encoded-image-to-a-numpy-array) site. I am getting base 64 screenshot and feed it in to io.BytsIO, however I get error message "a bytes-like object is required, not 'str'"
- When I encode it I get the following message. "cannot identify image file <_io.BytesIO object at 0x111db39e8>" [ref](https://stackoverflow.com/questions/42612002/python-sockets-error-typeerror-a-bytes-like-object-is-required-not-str-with/42612820)
- It seems that I need to understand better what is base64 what is string what does it represent. Will do some research
- Well I did not understand but instead of getting the screen shot as base 64 I have get the screenshot as png and used same code mentioned [here](https://stackoverflow.com/questions/57318892/convert-base64-encoded-image-to-a-numpy-array). And it worked!
- I have showed the series of images in a plot to see they are working fine. Yeah!!! That was quiker then expected...
- as long as I see This means I have completed the first [task](https://github.com/hakanonal/geodashml/projects/1#card-37027228) of my [project](https://github.com/hakanonal/geodashml/projects/1).
- the last image has FPS metric. I am not sure why it appeared.
- I am worried about frame rate of this loop. I mean every loop getting screenshot and converting to numpy array takes time. Moreover when we move this code to the real script there will be additional score check and traning process will involve. So every screenshot action will be longer. I  will call this frame rate: the time that passes between takining the screen shot and getting the result from the network. To get a littile bit sense of it I have increased the samlpe loop size. It does not seem promissing. If in the future game performance is not going to be increase I will consider if one of the reason may be the low frame rate. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt


options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=options)

try:
    driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
    wait = WebDriverWait(driver,timeout=10)
    wait.until(lambda d: d.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")) #wait until green flag appears

    flag_element = driver.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")
    flag_element.click()

    wait.until(lambda d: d.find_element_by_css_selector(".monitor_monitor-container_2J9gl")) #wait until score board appears

    driver.implicitly_wait(20)
    html = driver.find_element_by_tag_name("html")
    html.send_keys(Keys.SPACE)
    wait.until(lambda d: d.find_element_by_css_selector(".monitor_value_3Yexa")) #wait until score monitor element appears. 

    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 10))
    numpt_test_images = []
    for i in range(4):
        for j in range(4):
            image_png = driver.find_element_by_css_selector("#view").screenshot_as_png
            img = Image.open(io.BytesIO(image_png))
            numpt_test_images.append(np.asarray(img))
            axes[j][i].imshow(numpy_array)
            print(numpy_array.shape)
    plt.show()

finally:
    driver.quit()

## [Build Agent Class](https://github.com/hakanonal/geodashml/projects/1#card-37027254) 

29.04.2020 - Starting the agent class godspeed!

- My previous project (which is curentlly not on github) get me a lot of insight about reinfocement learning. So I will be copying a lot of code from there. Th emain structure was the play function to advance the step, choosing the strategy (exploit or explore) and get the next action and update the q-value accoring to new step. Also agent class should initilize the network.
- To test the agent class I am guessing that I am also going to look into the [Game Environment class task](https://github.com/hakanonal/geodashml/projects/1#card-37027280)
- So the network structure is very important. I can not remember right now but onve I have read an article that someone has made a super mario agent with convolutional network ending up with multiple dense layer and giving the prediction of action to give. In our example the action set very very easy press space or not. dude this is so simple. So our state is going to be the numpy array from the screen shot we get earlier and the action is going to be single nouron. To construct the model I need to figure out the shapes of the input. So I will do that in aboove section. Well the shape is (600,785,4) it is 4 channel picture. Very high quality I do not think that we need that much hish quality picture but we will see. It is going to be wild advanture.
- I am going to copy paste some code from my earlier learning sessions. My initial network will proablly overkill, but the idea is starting with convolutional layers following with pooling in the end flatten it with 2 dense layer and ending with single nuoron sigmoid layer. All middle dense layers are relu. This was genereally best pratice among the reading I have made so far. I may tweak the layer counts of conv and desse layers, nouron/filter sizes and shapes later. I think the padding should not be same because the input shape is already huge. It feels like it should be narrowed down quicklly.
- I little bit testing is going on if agent class is ok, how model summary looks like. Make some predictions with checking if class helper methods are working fine...
- I have copy paste the train and update methods from my previous project did some minor changes to fit it into this project accoring to the input and put shapes.
- Ok during re forming the train function I have relized that accoring to [q-learning algorithm](https://en.wikipedia.org/wiki/Q-learning) I need to take the max value of next action. Should my network's last layer must be single nouron or double neoron that one of them is the prob of not pressing space and other one is the prop of pressing the space key. Well according to [this](https://pathmind.com/wiki/deep-reinforcement-learning) page since we have only one action we need one nouron.
- However int eh train method I could not figure out where should use the action value in the q-value aupdate formula. thinking onit...
- appart from my last project I decided to introduce learning rate hypper parameter here...
- since there is only 1 action in action set, the action parameter in train method seems to be irrevalent. so I diceded to remove it. But I am not sure what is going to be consiquences. I think I will see that on [this](https://github.com/hakanonal/geodashml/projects/1#card-37027293) stage
- It seems that we have done with the agent class. To test if train method is working correctlly, I need to construct the [environment](https://github.com/hakanonal/geodashml/projects/1#card-37027280).

In [ ]:
import numpy as np
import random
from os import path
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Conv2DTranspose, BatchNormalization, UpSampling2D, Reshape
from keras.optimizers import Adam
from keras.models import load_model
from keras.activations import softmax


class agent:
    def __init__(self,discount=0.95,exploration_rate=0.9,decay_factor=0.9999, learning_rate=0.1):
        self.discount = discount # How much we appreciate future reward over current
        self.exploration_rate = exploration_rate # Initial exploration rate
        self.decay_factor = decay_factor
        self.learning_rate = learning_rate

        if(path.exists(self._getModelFilename())):
            self.model = load_model(self._getModelFilename())
        else:
            self.model = Sequential()
            self.model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(600,785,4)))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Conv2D(64, kernel_size=3, activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Conv2D(64, kernel_size=3, activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Flatten())
            self.model.add(Dense(64,activation="relu"))
            self.model.add(Dense(32,activation="relu"))
            self.model.add(Dense(1,activation="sigmoid"))
            self.model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    def _getModelFilename(self):
        return "geodash.h5"

    def get_next_action(self, state):
        if random.random() < self.exploration_rate: # Explore (gamble) or exploit (greedy)
            return self.random_action()
        else:
            return self.greedy_action(state)

    def greedy_action(self, state):
        return self.getQ(state) > 0.5
    def random_action(self):
        return random.random() > 0.5

    def getQ(self,state):
        state_to_predict = np.expand_dims(state,0)
        action_prediction = self.model.predict(state_to_predict)
        return action_prediction[0][0]

    def train(self, old_state, new_state, reward):
        
        old_state_prediction = self.getQ(old_state)
        new_state_prediction = self.getQ(new_state)

        old_state_prediction = ((1-self.learning_rate) * old_state_prediction) + (self.learning_rate * (reward + self.discount * new_state_prediction))

        x = np.expand_dims(old_state,0)
        y = np.expand_dims(old_state_prediction,0)
        self.model.fit(x,y,verbose=0)

    def update(self, old_state, new_state, reward):        
        self.train(old_state, new_state, reward)
        self.exploration_rate *= self.decay_factor


- I little bit testing is going on if agent class is ok, how model summary looks like. Make some predictions with checking if class helper methods are working fine...

In [ ]:
a = agent()

for i in range(len(numpt_test_images)):
    print("%f - %d - %d"%(
        a.getQ(numpt_test_images[i]),
        a.greedy_action(numpt_test_images[i]),
        a.get_next_action(numpt_test_images[i])
        ))


## [Build Environment](https://github.com/hakanonal/geodashml/projects/1#card-37027280)

29.04.2020
- My initial thoght is to make the environment to read the hyper parameters and also using the selenimum we can open game and start the main loop to play constantlly. we may also need to use selenium to read the reward(score) value which is already being shown on the screen. I should also decide the metrics to watch. so let's begin...
- currentlly I have incode the hyper parameters to see if the general stuture of the class is working. I will problly convert the configuration to get from wandb on [next](https://github.com/hakanonal/geodashml/projects/1#card-37027287) phase...
- I am thinking of how to use the episode parameter. The inner loop that lives as long as one game. So somehow I should detect if our here is going forward, he is crached or he is died. also read the reward.
- Ok second thoght, so I will continue the game loop as long as the reward is bigger than the reward that is readed one before. so if the reward is the same with the previus one then we have completed one episode(game). Let's try to code this.
- but before that I need to implement the selenium attributes to this class. I get a new error message from the driver. I am not sure whay I am getting that "HTTPConnectionPool(host='127.0.0.1', port=55773): Max retries exceeded with url: /session/c3a1e26d8e0b10a664932321d5ae2269/element (Caused by NewConnectionError(': Failed to establish a new connection: Errno 61 Connection refused'))" Before that I was not getting this error. I also need to handle the error stitation, otherwise a lot of chrome aplication will spawn. Now I need to leave, I will continue later on...

30.04.2020
- Let's see...
- Yes! well I did not understand why but the error gone. All I did was to restart the notebook.
- Ok where were we?
- Alright I have started the start method, start the game by pressing the space on html element. start a loop and succesfully read the score. The score is also being changed during the loop
- So the score is being read too fast. see (0,0,1,2,3,4,5,5,6,7,7,8,8,9,9,10,11,11,12,12,12,13,13,14,14,15,15,15,15) same reward may be repeat itself. However when the game is over, I see that the same reward repeats itself much more then others. So let's 5 consecutive repeat of the same reward can be declared as 1 episode. However this may behave differently in different environments. This part is subject for an improvement. 
- On the other hand, In my experiement I have only printed the reward. I will now try to get the screen shot and maybe sent it to agent prediction so that one loop will be computationally more heavy. I wonder if I see not only repeating reward I may not see some of the rewards. Let's see...
- Ok couple of wierd stuff is going on. First, I get the state with readState method and send it to agent's getQ method which should predict though network. I got a shape error. Because I see that when I do not hide the window then the resulation of the screen shot is changing and that's why the resulted numpy array has different shape. I need to transform the screen show every time :/, Other then that if I remove the getq from loop. I read wierd reward values when the window is hidden. I'll give one more try for that
- Ok I got it now! To debug I print the element's text one before the reward. I was expecting to see "Score" text because the way of the html. After some loop I started to get not score but "Player FPS" text. That text was recognized before (check my notes before above). The problem with these htmls they are marked as same class names but they do not have unique id. I need to construct more clever css selectors.
- I have read [this](https://stackoverflow.com/questions/2717480/css-selector-for-first-element-with-class) article. I have some idea the different between firts-child and first-type-of. I will try one them by modifying the readScore method. It did not work. FPS value still appear after some time.
- I need to see the full html when that FPS value appears. Since that FPS value appears only on hidden mode I need to code a test script. I did that by reading [this](https://www.browserstack.com/guide/get-html-source-of-web-element-in-selenium-webdriver)
- yep here is the full html after that. You see that the FPScomes to first position which is the problem.
```html
<div class="react-contextmenu-wrapper"><div class="monitor_monitor-container_2J9gl react-draggable box_box_2jjDp" style="touch-action: none; transform: translate(0px, 0px); top: 6px; left: 170px;"><div class="monitor_default-monitor_2vCcZ"><div class="monitor_row_2y_kM"><div class="monitor_label_ci1ok">Player: FPS</div><div class="monitor_value_3Yexa" style="background: rgb(255, 140, 26);">21</div></div></div></div></div><div class="react-contextmenu-wrapper"><div class="monitor_monitor-container_2J9gl react-draggable box_box_2jjDp" style="touch-action: none; transform: translate(0px, 0px); top: 6px; left: 5px;"><div class="monitor_default-monitor_2vCcZ"><div class="monitor_row_2y_kM"><div class="monitor_label_ci1ok">Score</div><div class="monitor_value_3Yexa" style="background: rgb(255, 140, 26);">15</div></div></div></div></div><div class="react-contextmenu-wrapper"><div class="monitor_monitor-container_2J9gl react-draggable box_box_2jjDp" style="touch-action: none; transform: translate(0px, 0px); top: 4px; left: 355px;"><div class="monitor_default-monitor_2vCcZ"><div class="monitor_row_2y_kM"><div class="monitor_label_ci1ok">Your Best</div><div class="monitor_value_3Yexa" style="background: rgb(255, 140, 26);">15</div></div></div></div></div>
```
- I need to distinguish these elements from the labels. [This](https://stackoverflow.com/questions/1520429/is-there-a-css-selector-for-elements-containing-certain-text) article says it is not possible. More research...
- The only attribute difference among the elements is the style that is repositionaning them. So maybe I can select via style attribute let give it try...
- Yep It worked. However even the only selecting that element itself seems to be heavy computational thing. Even I did not get the q-vlue via network it skips some of the rewards. And adding the q-value makes it even worse. I can get only 2 reward values in each game. :/. I am pretty sure that reading the reward with this much expensive is going to be problem in the future. This will lower the frame rate a lot. We can not expect the agent react fast enough. I need to find a clever way to read or keep track of the reward.

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from PIL import Image
import io
import numpy as np

class environment:
    def __init__(self):
        self.config = {
            'discount': 0.95,
            'exploration_rate': 0.9,
            'decay_factor': 0.9999,
            'learning_rate': 0.1,
            'episode': 100000,
            'hide_browser' : 1
        }
        self.agent = agent(
            discount=self.config['discount'],
            exploration_rate=self.config['exploration_rate'],
            decay_factor=self.config['decay_factor'],
            learning_rate=self.config['learning_rate']
            )
        self.max_reward = 0
        self.tot_reward = 0
        self.tot_penalty = 0
        self.tot_valid = 0
        self.episode = self.config['episode']
        
        self.browser_options = webdriver.ChromeOptions()
        if self.config['hide_browser']:
            self.browser_options.add_argument('headless')
        self.browser_driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=self.browser_options)
        self.browser_wait = WebDriverWait(self.browser_driver,timeout=10)

        self.browser_driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img"))
        flag_element = self.browser_driver.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")
        flag_element.click()
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".monitor_monitor-container_2J9gl"))

    def start(self):
        html_element = self.browser_driver.find_element_by_tag_name("html")
        html_element.send_keys(Keys.SPACE)
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".monitor_value_3Yexa"))
        while True:
            q_value = self.agent.getQ(self.readState())
            reward = 1#int(self.readScore())
            print('%f - %d'%(q_value,reward))

    def readScore(self):
        monitor_element = self.browser_driver.find_element_by_css_selector('.monitor-list_monitor-list-scaler_143tA .monitor_monitor-container_2J9gl[style="touch-action: none; transform: translate(0px, 0px); top: 6px; left: 5px;"] .monitor_value_3Yexa')
        return monitor_element.text

    def readState(self):
        image_png = self.browser_driver.find_element_by_css_selector("#view").screenshot_as_png
        img = Image.open(io.BytesIO(image_png))
        return np.asarray(img)



    def end(self):
        self.browser_driver.quit()

In [ ]:
e = environment()
try:
    e.start()
finally:
    e.end()

02.05.2020 - continue to develop environment

- I have decided to ignore the performance problem. Afterall I do notknow the impact of the reward reading process to the overall performance. If I see a performance problem because of low frame rate I will open a card for that. 
- So I am copy-paste all code both classes in the following section and continue on these versions. 
- Should I include a random seed hyperparameter for network initilization? Yeah why not? Nevermind it seems to be very complicated. I did not understand. I though it would be easy. However I will add these artices for future refercence. [This](https://machinelearningmastery.com/reproducible-results-neural-networks-keras/), [this](https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds) and [this](https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development)
- I will continue my first strategy: check the reward change to see if game is continuing...
- Ok I have completed the full environment. Theoritaclly of courese. However the single step in one episode is slow that I even can not detect if the episode has finisshed or not. I defintelly need new strategy or keeping a clever way to keep track of reward.
- The game flowwing though underneeth my program. My program's sight praticcly useless. I am kind of stuck here... Just looking into code...
- Ok I have a new idea. I have dicovered that there is P (pause) action. So in the loop before action is send We will pause game do the process preceed to the next state and unpause the game every time we play our action. This method will be tremendouslly slow but at least my program will be able to catch-up the game speed. the wait period between unpaouse and pause will be define our actual frame rate. Let'go
- Ok to move forward first I need to find the send the key "p" I will go back to the selenium discovery codes above
- Suprisinglly the sending key space to the html element does not seems to be working without headless mode. ı can not debug properlly. It is working inconsistantly. I have read the [wait](https://www.selenium.dev/documentation/en/webdriver/waits/) working principle the section implicit wait and I have understand that it is not waintg that many seconds but it sets the implicit wait parameter as that and use that wait every time we try to find en element. If I remove all implicit wait lines from the code this may improve our loop problem also. I will try that.
- The key presses are working fine in headless mode. It is working incosistant on without headless before start game it will be logical to wait with the time module to sleep some seconds.
- Headless or not sending key p to html element seems to be working for pausing the game. 
- I am still sturguggling the inconsistancies I try to understand what caouses in consistancies... Trying some random things going too fast can not write it all down. I am quiet mad right now...
- sending keys works totally incosistant. It works sometimes and sometimes it does not work. To continue this environmental problem must be solved.

In [ ]:
import numpy as np
import random
from os import path
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Conv2DTranspose, BatchNormalization, UpSampling2D, Reshape
from keras.optimizers import Adam
from keras.models import load_model
from keras.activations import softmax


class agent:
    def __init__(self,discount=0.95,exploration_rate=0.9,decay_factor=0.9999, learning_rate=0.1):
        self.discount = discount # How much we appreciate future reward over current
        self.exploration_rate = exploration_rate # Initial exploration rate
        self.decay_factor = decay_factor
        self.learning_rate = learning_rate

        if(path.exists(self._getModelFilename())):
            self.model = load_model(self._getModelFilename())
        else:
            self.model = Sequential()
            self.model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(600,785,4)))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Conv2D(64, kernel_size=3, activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Conv2D(64, kernel_size=3, activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            self.model.add(Flatten())
            self.model.add(Dense(64,activation="relu"))
            self.model.add(Dense(32,activation="relu"))
            self.model.add(Dense(1,activation="sigmoid"))
            self.model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    def _getModelFilename(self):
        return "geodash.h5"

    def get_next_action(self, state):
        if random.random() < self.exploration_rate: # Explore (gamble) or exploit (greedy)
            return self.random_action()
        else:
            return self.greedy_action(state)

    def greedy_action(self, state):
        return self.getQ(state) > 0.5
    def random_action(self):
        return random.random() > 0.5

    def getQ(self,state):
        state_to_predict = np.expand_dims(state,0)
        action_prediction = self.model.predict(state_to_predict)
        return action_prediction[0][0]

    def train(self, old_state, new_state, reward):
        
        old_state_prediction = self.getQ(old_state)
        new_state_prediction = self.getQ(new_state)

        old_state_prediction = ((1-self.learning_rate) * old_state_prediction) + (self.learning_rate * (reward + self.discount * new_state_prediction))

        x = np.expand_dims(old_state,0)
        y = np.expand_dims(old_state_prediction,0)
        self.model.fit(x,y,verbose=0)

    def update(self, old_state, new_state, reward):        
        self.train(old_state, new_state, reward)
        self.exploration_rate *= self.decay_factor



from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from PIL import Image
import io
import numpy as np
import time

class environment:
    def __init__(self):
        self.config = {
            'discount': 0.95,
            'exploration_rate': 0.9,
            'decay_factor': 0.9999,
            'learning_rate': 0.1,
            'episode': 100000,
            'hide_browser' : 0
        }
        self.agent = agent(
            discount=self.config['discount'],
            exploration_rate=self.config['exploration_rate'],
            decay_factor=self.config['decay_factor'],
            learning_rate=self.config['learning_rate']
            )
        self.reward = 0
        self.max_reward = 0
        self.tot_reward = 0
        self.tot_penalty = 0
        self.tot_valid = 0
        self.episode = self.config['episode']
        
        self.browser_options = webdriver.ChromeOptions()
        if self.config['hide_browser']:
            self.browser_options.add_argument('headless')
        self.browser_driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=self.browser_options)
        self.browser_wait = WebDriverWait(self.browser_driver,timeout=10)

        self.browser_driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img"))
        flag_element = self.browser_driver.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")
        flag_element.click()
        time.sleep(10)
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".monitor_monitor-container_2J9gl"))
        self.browser_html_element = self.browser_driver.find_element_by_tag_name("html")

    def start(self):
        self.playAction(1) #to start the game
        self.browser_wait.until(lambda d: d.find_element_by_css_selector(".monitor_value_3Yexa"))        
        self.browser_html_element.send_keys("p")
        
        for episode in range(1,self.config['episode']+1):
            self.reward = 0
            past_reward = 0
            while True:
                old_state = self.readState()
                action_to_play = self.agent.get_next_action(old_state)
                self.playAction(action_to_play)
                new_state = self.readState()
                self.reward = int(self.readScore())
                self.agent.update(old_state,new_state,self.reward)
                self.tot_reward += self.reward
                self.max_reward = max(self.max_reward,self.reward)
                if(self.reward == past_reward):
                    self.tot_penalty += 1
                    break
                else:
                    self.tot_valid += 1
                past_reward = self.reward
            
                metrics = {
                    'valid-total' : self.tot_valid,
                    'valid-avarage' : self.tot_valid/episode,
                    'penalty-total' : self.tot_penalty,
                    'penalty-avarage' : self.tot_penalty/episode,
                    'exploration-rate' : self.agent.exploration_rate,
                    'max-reward' : self.max_reward,
                    'tot-reward' : self.tot_reward,
                    'avg-reward' : self.tot_reward/episode,
                    'reward':self.reward,
                }
                print(metrics)

    def readScore(self):
        monitor_element = self.browser_driver.find_element_by_css_selector('.monitor-list_monitor-list-scaler_143tA .monitor_monitor-container_2J9gl[style="touch-action: none; transform: translate(0px, 0px); top: 6px; left: 5px;"] .monitor_value_3Yexa')
        return monitor_element.text

    def readState(self):
        image_png = self.browser_driver.find_element_by_css_selector("#view").screenshot_as_png
        img = Image.open(io.BytesIO(image_png))
        if(self.config['hide_browser'] == 0):
            img = img.resize((785,600), Image.ANTIALIAS)
        return np.asarray(img)

    def playAction(self,action):
        self.browser_html_element.send_keys("p") #unpause
        if action:
            self.browser_html_element.send_keys(Keys.SPACE)
        self.browser_html_element.send_keys("p") #pause
            


    def end(self):
        self.browser_driver.quit()

In [ ]:
e = environment()
try:
    e.start()
finally:
    e.end()

### [Inconsistant key sends](https://github.com/hakanonal/geodashml/projects/1#card-37424332)

04.05.2020 - New day brings new energy...

- First I will check if there is an alternative for selenium. [Mechanize](https://mechanize.readthedocs.io/en/latest/index.html) may be an option. But before moving forward I want to spent some more time to staring the last code that I have written so far.
- What about trying different element to send keys. how about body. or something else. Moreover I must find a way to prove that the keys are actually being send %100 percent every time.
- body element also seems to reponse to key sends but again it is not consistant.
- while reading [this](https://www.selenium.dev/documentation/en/webdriver/web_element/) I have see there is switch_to directive. Maybe it is loosing some kind of focus/activeness that's way key sends may be lost in between. I will play arround a little bit in this area.
- One more idea in this [keyboard](https://www.selenium.dev/documentation/en/webdriver/keyboard/) page it gives example about keydown and keyup. Maybe sending the space key is since it is side key not a letter it should be used with keyup and keydown. by the way starting from begening of the game if I keep pressing the space bar I go way too much in the game. I will try to keep pressing key. After that I will try actions as mentioned in the [document](https://www.selenium.dev/documentation/en/webdriver/keyboard/) 
- Well pressing the space key all the time seems to be working. And eventhoug I did not give an element to press.
- I will try key_down and key_up againto simulate sending the space key
- I have converted the p key to action. However space key to start the game did nor work this time.
- I will try this activeness thing...
- I am currentlly trying almost all methods and attributes of the selenium library. doing lot's of things that I do not have patience to write it down.
- I have spent way to much time on this stupid thing, where we are very far away from project's goal.
- By reading [this](https://mechanize.readthedocs.io/en/latest/faq.html#javascript-is-messing-up-my-web-scraping-what-do-i-do) ı immediatlly this won't work for me also. because I will not be able to execute javasciprt nor scrath.
- So far in out project we did not come to a point that we did not start to train our model yet. I am still trying to solidized the environment. So I am stuck here. If you have any idea it would be much appricaited from this point.

In [1]:
from selenium import webdriver
from IPython.display import Image
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

options = webdriver.ChromeOptions()
#options.add_argument('headless')
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", chrome_options=options)

try:
    driver.get('https://scratch.mit.edu/projects/105500895/fullscreen/')
    wait = WebDriverWait(driver,timeout=10)
    print("init")
    wait.until(lambda d: d.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv img")) #wait until green flag appears
    print("green flag loaded...")

    flag_element = driver.find_element_by_css_selector(".stage_green-flag-overlay_gNXnv")
    wait.until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR,".stage_green-flag-overlay_gNXnv")))
    flag_element.click()
    print("green flag clicked...")
    print("waiting...")
    time.sleep(10)
    print("wait done!")

    body = driver.find_element_by_tag_name("body")
    #canvas_div = driver.find_element_by_css_selector(".stage_stage_1fD7k")
    #wait.until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR,".stage_stage_1fD7k")))
    #canvas_div.click()
    space_action = webdriver.ActionChains(driver).key_down(Keys.SPACE).key_up(Keys.SPACE)
    space_action.perform()
    print("started the game")
    wait.until(lambda d: d.find_element_by_css_selector(".monitor_value_3Yexa")) #wait until score monitor element appears. 

    flag = 1
    for i in range(100):
        print(i)
        driver.switch_to_window(driver.current_window_handle)
        body.send_keys("p")
        if flag == 1:
            print("paused")
        else:
            print("unpaused")
        flag *= -1            
        time.sleep(1)

finally:
    #print("ok")
    driver.quit()

init
green flag loaded...
green flag clicked...
waiting...
wait done!
started the game
0
paused
1
unpaused
2
paused
3
unpaused
4
paused
5
unpaused
6
paused
7
unpaused
8
paused
9
unpaused
10
paused
11
unpaused
12
paused
13
unpaused
14
paused
15
unpaused
16
paused
17
unpaused
18
paused
19
unpaused
20
paused
21
unpaused
22
paused
23
unpaused
24
paused
25
unpaused
26
paused
27
unpaused
28
paused
29
unpaused
30
paused
31
unpaused
32
paused
33
unpaused
34
paused
35
unpaused
36
paused
37
unpaused
38
paused
39
unpaused
40
paused
41
unpaused
42
paused
43
unpaused
44
paused
45
unpaused
46
paused
47
unpaused
48
paused
49
unpaused
50
paused
51
unpaused
52
paused
53
unpaused
54
paused
55
unpaused
56
paused
57
unpaused
58
paused
59
unpaused
60
paused
61
unpaused
62
paused
63
unpaused
64
paused
65
unpaused
66
paused
67
unpaused
68
paused
69
unpaused
70
paused
71
unpaused
72
paused
73
unpaused
74
paused
75
unpaused
76
paused
77
unpaused
78
paused
79
unpaused
80
paused
81
unpaused
82
paused
83
unpaused